This model is extremely RAM-intensive, and quickly consumes all my memory in laptop as well as Google Colab causing the kernels to crash repeatedly. Due to hardware constraints, I have prepared a small dataset- of 1000 offensive and non-offensive texts along with their descriptions. They are the first 1000 (non offensive) and the last 1000 (offensive) values from the caption_description dataframe prepared earlier. The following document involves a sample code which was used to make the dataset, but it will not run without crashing. 

In [1]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
import os
import time

In [2]:
main_df= pd.read_pickle("../outputs/caption_description.pkl")
main_df

,label,text,file,description,sets
0,0,berserk 2016 is a good adaptation you're kidd...,71094.png,woman with a monkey mask and a fake monkey,[test_unseen]
1,0,my life goal? make somebody this fucking trig...,91724.png,woman holding a cigarette in her hand,[train]
2,0,""" i don't wanna, just get it, get it, get it, ...",64280.png,man wearing a hat and a tie,[train]
3,0,"""1st day of 4th grade sandy hook elementary sc...",67082.png,group of children standing in front of a schoo...,"[dev_seen, dev_unseen]"
4,0,"""a blow job a day will keep his side chicks aw...",46380.png,woman with a black top and a blue background,[train]
...,...,...,...,...,...
12135,1,your post is under coon review and it ain't lo...,59482.png,man speaking into a microphone,[test_unseen]
12136,1,your purchase of $19.99 comes to $21.36 after ...,21693.png,man in a kitchen with a calculator,[train]
12137,1,your purchase of $19.99 comes to $21.36 after ...,59837.png,man in a kitchen with a calculator,[train]
12138,1,"your sense of humor is so dark, police want to...",42897.png,man talking into a microphone,[train]


In [3]:
text= main_df["text"]

In [9]:
max([len(tokenizer.tokenize(sentence)) for sentence in main_df["text"]])

86

In [ ]:
vals=[]
attention_masks=[]
for sentence in text:
    vals.append(['[CLS]']+tokenizer.tokenize(sentence)+['[SEP]'])
max_pad= max([len(val) for val in vals])

for val in vals:
    pad_count= max_pad-len(val)
    attention_masks.append([1]*len(val)+[0]*pad_count)
    if(len(val)!=max_pad):
        val.extend(['[PAD]']*pad_count) 

In [ ]:
token_ids=[]
for val in vals:
    token_ids.append(tokenizer.convert_tokens_to_ids(val))
token_ids= torch.tensor(token_ids)
attention_masks= torch.tensor(attention_masks)

In [ ]:
batch_size=50

In [ ]:
num_batches = len(token_ids) // batch_size
if len(token_ids) % batch_size != 0:
    num_batches += 1
sentence_embeddings=[]

num_batches

243

In [ ]:
# for i in range(num_batches):
#     start_index = i * batch_size
#     end_index = min((i + 1) * batch_size, len(token_ids))

#     # Extract a batch of token IDs and attention masks
#     token_ids_batch = token_ids[start_index:end_index]
#     attention_masks_batch = attention_masks[start_index:end_index]

#     # Convert to tensors
#     token_ids_tensor = torch.tensor(token_ids_batch)
#     attention_masks_tensor = torch.tensor(attention_masks_batch)

#     # Forward pass through BERT model
#     output = model(token_ids_tensor, attention_mask=attention_masks_tensor)
#     sentence_embeddings.append(output[1])